# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [39]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

[nltk_data] Downloading package punkt to /home/rajat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/rajat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/rajat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
import sklearn
sklearn.__version__

'0.20.3'

In [2]:
# load data from database
engine = create_engine('sqlite:///data/InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# independant variable (X), dependent variable (Y), category names
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
categories = Y.columns.tolist()

In [4]:
list(categories)

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = text.lower()

    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    stop_words = nltk.corpus.stopwords.words("english")
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
print(X_train.shape, y_train.shape)

(19662,) (19662, 36)


In [8]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
from sklearn.metrics import classification_report
test_pred = pipeline.predict(X_test)

In [10]:
for i in range(len(categories)):
    print(categories[i])
    print(classification_report(y_test[categories[i]], test_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.60      0.14      0.23      1531
           1       0.78      0.97      0.86      4977
           2       0.42      0.24      0.31        46

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.60      0.45      0.47      6554
weighted avg       0.74      0.77      0.71      6554

request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      5410
           1       0.78      0.50      0.61      1144

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.84      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

In [11]:
y_train.shape

(19662, 36)

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__base_estimator', 'clf__estimator__learning_rate', 'clf__estimator__n_estimators', 'clf__estimator__random_state', 'clf__estimator', 'clf__n_jobs'])

In [13]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.8, 1.0),
    'vect__max_features': (None, 10000),
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate': [0.1, 1.0]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)

In [14]:
cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.8, 1.0), 'vect__max_features': (None, 10000), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [0.1, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
test_pred = cv.predict(X_test)

In [16]:
for i in range(len(categories)):
    print(categories[i])
    print(classification_report(y_test[categories[i]], test_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.60      0.15      0.24      1531
           1       0.78      0.97      0.87      4977
           2       0.42      0.30      0.35        46

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.60      0.47      0.49      6554
weighted avg       0.74      0.77      0.72      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5410
           1       0.77      0.53      0.63      1144

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.84      0.75      0.78      6554
weighted avg       0.88      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.08      0.03      0.05        31

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.54      0.52      0.52      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [18]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [19]:
parameters2 = {    
            'clf__estimator__n_estimators': [50, 100],
            'clf__estimator__min_samples_split': [2, 4]
}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2)

In [20]:
cv2.fit(X_train, y_train)

/home/rajat/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
test_pred = cv2.predict(X_test)

In [29]:
for i in range(len(categories)):
    print(categories[i])
    print(classification_report(y_test[categories[i]], test_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.73      0.34      0.46      1531
           1       0.82      0.96      0.88      4977
           2       0.78      0.15      0.25        46

   micro avg       0.81      0.81      0.81      6554
   macro avg       0.78      0.48      0.53      6554
weighted avg       0.80      0.81      0.78      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5410
           1       0.88      0.46      0.60      1144

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.89      0.72      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      655

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6271
           1       0.25      0.00      0.01       283

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.60      0.50      0.49      6554
weighted avg       0.93      0.96      0.94      6554

weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4719
           1       0.86      0.66      0.74      1835

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.87      0.81      0.83      6554
weighted avg       0.87      0.87      0.87      6554

floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6000
           1       0.93      0.43      0.58       554

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.94      0.71      0.78      6554
weighted avg       0.95      0.95      0.94      65

/home/rajat/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:
cv.best_params_

{'clf__estimator__learning_rate': 1.0,
 'clf__estimator__n_estimators': 100,
 'vect__max_df': 1.0,
 'vect__max_features': 10000,
 'vect__ngram_range': (1, 2)}

In [31]:
cv2.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}

### 9. Export your model as a pickle file

In [36]:
import joblib

In [37]:
joblib.dump(cv.best_estimator_, 'disaster_model.pkl')

['disaster_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.